# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv('Output_data/cities.csv')
city_weather.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Karrāpur,23.9500,78.8500,87.49,20,17,9.93,IN,1651352930
1,1,Severo-Kuril'sk,50.6789,156.1250,35.17,79,100,9.53,RU,1651352930
2,2,Ushuaia,-54.8000,-68.3000,40.66,75,0,0.00,AR,1651352864
3,3,New Norfolk,-42.7826,147.0587,56.91,69,100,7.00,AU,1651352931
4,4,Rikitea,-23.1203,-134.9692,76.03,75,52,10.67,PF,1651352932


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]
locations

,Lat,Lng
0,23.9500,78.8500
1,50.6789,156.1250
2,-54.8000,-68.3000
3,-42.7826,147.0587
4,-23.1203,-134.9692
...,...,...
529,23.2955,113.8284
530,51.6667,135.7000
531,19.9333,-105.2500
532,42.4308,130.6434


In [4]:
humidity

0      20
1      79
2      75
3      69
4      75
       ..
529    94
530    85
531    28
532    62
533    77
Name: Humidity, Length: 534, dtype: int64

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
fit_weather_data = city_weather.loc[(city_weather['Wind Speed']<= 10) & (city_weather['Cloudiness'] == 0) & 
                                    (city_weather['Max Temp']<=80) & (city_weather['Max Temp'] >= 70)]
fit_weather_data.dropna()
fit_weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
189,189,Riyadh,24.6877,46.7219,75.34,26,0,5.91,SA,1651352826
286,286,Mahajanga,-15.7167,46.3167,78.17,79,0,8.70,MG,1651353089
385,385,Winslow,35.0242,-110.6974,75.38,7,0,3.44,US,1651353143


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = fit_weather_data.loc[:,:]
hotel_df['Hotel Name'] = ""

target_type = "hotel"
radius = 5000

# rewrite params dict

params = {
    "keyword" : "hotel",
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    city_name = row["City"]

    params["location"] = f"{row['Lat']}, {row['Lng']}"
    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
   #  time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

C:\Users\alisa\AppData\Local\Temp/ipykernel_21576/1509460459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


Retrieving Results for Index 189: Riyadh.
Closest hotel in Riyadh is Radisson Blu Hotel, Riyadh.
------------
Retrieving Results for Index 286: Mahajanga.


C:\Users\alisa\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel in Mahajanga is Baobab Tree Hotel & Spa.
------------
Retrieving Results for Index 385: Winslow.
Closest hotel in Winslow is La Posada Hotel.
------------
-------End of Search-------


In [8]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
189,189,Riyadh,24.6877,46.7219,75.34,26,0,5.91,SA,1651352826,"Radisson Blu Hotel, Riyadh"
286,286,Mahajanga,-15.7167,46.3167,78.17,79,0,8.70,MG,1651353089,Baobab Tree Hotel & Spa
385,385,Winslow,35.0242,-110.6974,75.38,7,0,3.44,US,1651353143,La Posada Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))